<a href="https://colab.research.google.com/github/LouisKoXiang/Wits-Hackathon/blob/main/2025%E9%BB%91%E5%AE%A2%E6%9D%BE%E6%B5%81%E7%A8%8Bdemo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
# =========================================
# Lending Club Risk Predictor Prototype
# =========================================
# Author: Louis Ko
# Description:
#   - ML + FastAPI + LangChain + ngrok 一鍵整合示範
#   - 黑客松原型架構，可快速擴充為完整信貸風險預測系統
#   - 可依角色分工（ML / API / Frontend）
# =========================================


# =========================================
# STEP 1 安裝套件
# =========================================
!pip install fastapi uvicorn pyngrok tensorflow scikit-learn pandas numpy seaborn matplotlib joblib langchain openai --quiet


# =========================================
# STEP 2 建立模擬資料與模型（可替換為真實數據）
# =========================================
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import joblib, json, os

# -----------------------------------------
# 模擬假資料（範例資料用於快速測試模型功能）
# TODO: 改用真實 Lending Club / 內部信貸資料
# -----------------------------------------
np.random.seed(42)
n = 1000
df = pd.DataFrame({
    'loan_amnt': np.random.randint(5000, 40000, n),    # 貸款金額
    'annual_inc': np.random.randint(30000, 200000, n), # 年收入
    'dti': np.random.uniform(5, 40, n),                # 債務比 (Debt-to-Income ratio)
    'total_acc': np.random.randint(5, 40, n),          # 帳戶總數
    'revol_util': np.random.uniform(0, 100, n),        # 循環信貸使用率
    'int_rate': np.random.uniform(5, 25, n),           # 利率 (%)
    'loan_repaid': np.random.choice([0, 1], n, p=[0.25, 0.75])  # 是否繳清（0=違約，1=已繳清）
})

# -----------------------------------------
# ML 組任務：可替換資料集後延伸特徵工程
# e.g. One-Hot Encoding、Log Transformation、Feature Selection
# -----------------------------------------
X = df.drop('loan_repaid', axis=1).values
y = df['loan_repaid'].values

scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=101)

# -----------------------------------------
# 簡易神經網路模型（可替換為更強模型）
# TODO: 改為 XGBoost / LightGBM / RandomForest / Deep NN
# -----------------------------------------
model = tf.keras.Sequential([
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(8, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam')
model.fit(X_train, y_train, epochs=5, batch_size=32, validation_data=(X_test, y_test), verbose=1)

# =========================================
# STEP 3 儲存模型與前處理器
# =========================================
os.makedirs("/content/model", exist_ok=True)
model.save("/content/model/LendingClub.keras")
joblib.dump(scaler, "/content/model/scaler.pkl")

columns = df.drop('loan_repaid', axis=1).columns.tolist()
with open("/content/model/columns.json", "w") as f:
    json.dump(columns, f)

print("模型與前處理器已儲存完畢。")


# =========================================
# STEP 4 FastAPI 服務（API 組）
# =========================================
# 提供 /predict 端點，供前端或 Agent 呼叫
# -----------------------------------------
api_code = r"""
from fastapi import FastAPI
import tensorflow as tf
import joblib, json, numpy as np

app = FastAPI()

model = tf.keras.models.load_model("/content/model/LendingClub.keras")
scaler = joblib.load("/content/model/scaler.pkl")
with open("/content/model/columns.json") as f:
    columns = json.load(f)

@app.post("/predict")
def predict(data: dict):
    X = np.array([[data.get(col, 0) for col in columns]])
    X_scaled = scaler.transform(X)
    pred = model.predict(X_scaled)[0][0]

    risk_level = "Low" if pred > 0.8 else "Medium" if pred > 0.4 else "High"

    return {
        "risk_score": float(pred),
        "risk_level": risk_level,
        "model_columns": columns
    }
"""

os.makedirs("/content/api", exist_ok=True)
with open("/content/api/main.py", "w") as f:
    f.write(api_code)

print("FastAPI 服務程式建立完成。")


# =========================================
# STEP 5 啟動 FastAPI + ngrok（後端通訊層）
# =========================================
from pyngrok import ngrok
import requests, time

ngrok.kill()
print("舊 ngrok 通道已清除。")

# TODO: 換成團隊 ngrok authtoken（共用一組帳號）
ngrok.set_auth_token("your_ngrok_token_here")

get_ipython().system_raw("uvicorn api.main:app --host 0.0.0.0 --port 8000 &")

public_url = ngrok.connect(8000)
print("Public API URL:", public_url.public_url)

for i in range(15, 0, -3):
    print(f"等待伺服器啟動中...（{i} 秒）")
    time.sleep(3)


# =========================================
# STEP 6 測試 /predict API
# =========================================
sample = {
    "loan_amnt": 25000,    # 貸款金額 Loan Amount — 客戶申請的貸款金額（單位：美元）
    "annual_inc": 100000,  # 年收入 Annual Income — 客戶的年收入，用以評估還款能力
    "dti": 25.5,           # 債務比 Debt-To-Income Ratio — 每月債務支付金額佔月收入的比例（%）
    "total_acc": 15,       # 帳戶總數 Total Accounts — 客戶在金融機構的帳戶總數（信用卡、房貸、車貸等）
    "revol_util": 42.3,    # 循環信貸使用率 Revolving Utilization — 信用卡餘額相對於額度的使用比例（%）
    "int_rate": 10.5       # 利率 Interest Rate — 貸款利率，通常風險越高利率越高
}

url = public_url.public_url + "/predict"

try:
    r = requests.post(url, json=sample, timeout=10)
    print("Status:", r.status_code)
    print("Text:", r.text)
    if r.headers.get("content-type", "").startswith("application/json"):
        print("JSON:", r.json())
    else:
        print("非 JSON 回傳，可能仍在啟動中。請再執行一次。")
except Exception as e:
    print("請求失敗:", e)


# =========================================
# STEP 7 LangChain Agent 工具層（AI 組）
# =========================================
# 提供 LangChain Tool，讓 Agent 能以自然語言呼叫風險預測 API
from langchain.tools import tool

@tool
def risk_predict_tool(customer_info: dict):
    """呼叫貸款風險預測 API"""
    import requests
    response = requests.post(url, json=customer_info)
    return response.json()

print("\nAgent Tool 測試：")
print(risk_predict_tool.invoke({"customer_info": sample}))


Epoch 1/5
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.6972 - val_loss: 0.6827
Epoch 2/5
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.6711 - val_loss: 0.6617
Epoch 3/5
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.6408 - val_loss: 0.6355
Epoch 4/5
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.6100 - val_loss: 0.6167
Epoch 5/5
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.5704 - val_loss: 0.6170
模型與前處理器已儲存完畢。
FastAPI 服務程式建立完成。
舊 ngrok 通道已清除。


ERROR:pyngrok.process.ngrok:t=2025-11-08T10:55:41+0000 lvl=eror msg="failed to reconnect session" obj=tunnels.session err="failed to dial ngrok server with address \"connect.us.ngrok-agent.com:443\": dial tcp 3.148.88.97:443: i/o timeout"


Public API URL: https://22e6925da030.ngrok-free.app
等待伺服器啟動中...（15 秒）
等待伺服器啟動中...（12 秒）
等待伺服器啟動中...（9 秒）
等待伺服器啟動中...（6 秒）
等待伺服器啟動中...（3 秒）
Status: 200
Text: {"risk_score":0.6777337193489075,"risk_level":"Medium","model_columns":["loan_amnt","annual_inc","dti","total_acc","revol_util","int_rate"]}
JSON: {'risk_score': 0.6777337193489075, 'risk_level': 'Medium', 'model_columns': ['loan_amnt', 'annual_inc', 'dti', 'total_acc', 'revol_util', 'int_rate']}

Agent Tool 測試：
{'risk_score': 0.6777337193489075, 'risk_level': 'Medium', 'model_columns': ['loan_amnt', 'annual_inc', 'dti', 'total_acc', 'revol_util', 'int_rate']}
